# Summarize large text

<p style="font-size: 20px; font-weight: bold; font-style: italic;">...via LLM-graph</p>

Anton Antonov   
August 2025

----

## Introduction

This notebook illustrates how to specify a Large Language Model (LLM) graph for deriving comprehensive summaries of large texts. 
The LLM graph is based on different LLM- and non-LLM functions .
The Raku package ["LLM::Graph"](https://raku.land/zef:antononcube/LLM::Graph) is used, [AAp1].

Using the LLM graph is an alternative to the Literate programming based solutions shown in [AA1, AAn1].

----

## Setup

In [2]:
use LLM::Graph;
use Data::Importers;

use LLM::Tooling;

In [ ]:
sink my $conf5-mini = llm-configuration('ChatGPT', model => 'gpt-5-mini');
sink my $conf41-mini = llm-configuration('ChatGPT', model => 'gpt-4.1-mini', temperature => 0.55, max-tokens => 4096)

----

## LLM graph

Specify the LLM graph nodes:

In [5]:
sink my %rules =
TypeOfInput => sub ($_) {
        "Determine the input type of\n\n$_.\n\nThe result should be one of: 'Text', 'URL', 'FilePath', or 'Other'."  ~ 
        llm-prompt('NothingElse')('single string')
    },

IngestText =>  { eval-function => sub ($TypeOfInput, $_) { $TypeOfInput ~~ / URL | FilePath/ ?? data-import($_) !! $_} },

Title => { 
    eval-function => sub ($IngestText, $with-title = Whatever) { $with-title ~~ Str:D ?? $with-title !! llm-synthesize([llm-prompt("TitleSuggest")($IngestText, 'article'), "Short title with less that 6 words"]) },
},

Summary => sub ($IngestText) { llm-prompt("Summarize")() ~ "\n\n$IngestText" },

TopicsTable => sub ($IngestText) { llm-prompt("ThemeTableJSON")($IngestText, 'article', 20) },

ThinkingHats => sub ($IngestText) { llm-prompt("ThinkingHatsFeedback")($IngestText, 'HTML') },

MindMap => sub ($IngestText) { 
    [
        "Create a concise mermaid-js mind-map for the text:",
        $IngestText,
        "All node messages should be in quotes.",
        llm-prompt("NothingElse")("correct mermaid-js")
    ]
},

Report => { eval-function => 
    sub ($Title, $Summary, $TopicsTable, $MindMap, $ThinkingHats) { 
        [
            "# $Title",
            '### *LLM summary report*',
            '## Summary',
            $Summary,
            '## Topics',
            to-html(
                from-json($TopicsTable.subst(/ ^ '```json' | '```' $/):g),
                field-names => <theme content>,
                align => 'left'),
            "## Mind map",
            $MindMap,
            '## Thinking hats',
            $ThinkingHats.subst(/ ^ '```html' | '```' $/):g
        ].join("\n\n")
    } 
},

ExportAndOpen => {
    eval-function => sub ($Report) {
       spurt('./Report.md', $Report);
       shell "open ./Report.md" 
    },
    test-function => -> $export-and-open = True { $export-and-open ~~ Bool:D && $export-and-open || $export-and-open.Str.lc ∈ <true yes open> }
}
;

Make the graph:

In [ ]:
my $gCombinedSummary = LLM::Graph.new(%rules, llm-evaluator => $conf5-mini, :async)

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

----

## Full computation

URL and text statistics:

In [7]:
my $url = 'https://raw.githubusercontent.com/antononcube/RakuForPrediction-blog/refs/heads/main/Data/Graph-neat-examples-in-Raku-Set-2-YouTube.txt';
my $txtFocus = data-import($url);

text-stats($txtFocus)

(chars => 5957 words => 1132 lines => 157)

Computation:

In [8]:
$gCombinedSummary.eval($url, with-title => '«Graph» neat examples, set 3')

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

**Remark:** Instead of deriving the title using an LLM, the title is specified as an argument.

Show the corresponding graph:

In [17]:
#% html
$gCombinedSummary.dot(node-width => 1.2, theme => 'ortho'):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- $_ -->
 
 $_ 
 
 $_ 
 
<!-- IngestText -->
 
 IngestText 
 
 IngestText 
 
<!-- $_->IngestText -->
 
 $_->IngestText 
 
 
 
<!-- TypeOfInput -->
 
 TypeOfInput 
 
 TypeOfInput 
 
<!-- $_->TypeOfInput -->
 
 $_->TypeOfInput 
 
 
 
<!-- ExportAndOpen -->
 
 ExportAndOpen 
 
 ExportAndOpen 
 
<!-- MindMap -->
 
 MindMap 
 
 MindMap 
 
<!-- IngestText->MindMap -->
 
 IngestText->MindMap 
 
 
 
<!-- Summary -->
 
 Summary 
 
 Summary 
 
<!-- IngestText->Summary -->
 
 IngestText->Summary 
 
 
 
<!-- ThinkingHats -->
 
 ThinkingHats 
 
 ThinkingHats 
 
<!-- IngestText->ThinkingHats -->
 
 IngestText->ThinkingHats 
 
 
 
<!-- Title -->
 
 Title 
 
 Title 
 
<!-- IngestText->Title -->
 
 IngestText->Title 
 
 
 
<!-- TopicsTable -->
 
 TopicsTable 
 
 TopicsTable 
 
<!-- IngestText->TopicsTable -->
 
 IngestText->TopicsTable 
 
 
 
<!-- Report -->
 
 Report 
 
 Report 
 
<!-- MindMap->Report -->
 
 MindMap->Report 
 
 
 
<!-- Report->ExportAndOpen -->
 
 Report->ExportAndOpen 
 
 
 
<!-- Summary->Report -->
 
 Summary->Report 
 
 
 
<!-- ThinkingHats->Report -->
 
 ThinkingHats->Report 
 
 
 
<!-- Title->Report -->
 
 Title->Report 
 
 
 
<!-- TopicsTable->Report -->
 
 TopicsTable->Report 
 
 
 
<!-- TypeOfInput->IngestText -->
 
 TypeOfInput->IngestText

Final result:

In [10]:
#% markdown
$gCombinedSummary.nodes<Report><result>.subst(/'```html' | '```' $/):g

# Visualizing Graphs with Raku

### *LLM summary report*

## Summary

The presentation titled "Graph Neat Examples in Raku" by Anton Antonov demonstrates various graph visualizations using the Raku programming language and its graph package, including undirected triangular grid graphs, bipartite grid graphs, directed graphs with connected components, and collages of star graphs. Visualization methods include JavaScript D3-based rendering and SVG generation via the DOT language from Graphviz, which allows flexible styling and layout options. The examples highlight concise Raku code that showcases graph properties, computational possibilities, and integration with web-based tools like Jupyter notebooks.

## Topics

<table border="1"><thead><tr><th>theme</th><th>content</th></tr></thead><tbody><tr><td align=left>Introduction and Overview</td><td align=left>Presentation title, date, presenter name, and introduction to undirected triangular grid graph example.</td></tr><tr><td align=left>Graph Matrices Explanation</td><td align=left>Explanation of adjacency and incidence matrices for undirected graphs with vertex-edge relationships.</td></tr><tr><td align=left>NE Examples Definition</td><td align=left>Definition and purpose of NE examples showcasing Raku programming and computational possibilities.</td></tr><tr><td align=left>Graph Visualization Tools</td><td align=left>Using Raku graph package, JavaScript D3, and GraphViz dot language for graph visualization.</td></tr><tr><td align=left>Bipartite Grid Graph Example</td><td align=left>Demonstration of bipartite property in grid graphs with vertex coloring and bipartite verification.</td></tr><tr><td align=left>Code Walkthrough for Bipartite Graph</td><td align=left>Explanation of code generating grid graph, verifying bipartiteness, and applying vertex highlights.</td></tr><tr><td align=left>Directed Grid Graph and Connected Components</td><td align=left>Randomly directed grid graph example illustrating connected components and reachability.</td></tr><tr><td align=left>Visualization Methods in Jupyter</td><td align=left>Rendering JavaScript and SVG graph visualizations in Jupyter notebooks using different methods.</td></tr><tr><td align=left>Dot Language Usage and Node Shapes</td><td align=left>Using dot language for graph styling with random node shapes and terminology preferences.</td></tr><tr><td align=left>Star Graphs Collage Example</td><td align=left>Creating and visualizing multiple star graphs with varied rays and colors.</td></tr><tr><td align=left>Dot Language Advantages</td><td align=left>Benefits of using dot DSL for styling and specifying graph elements efficiently.</td></tr><tr><td align=left>Conclusion</td><td align=left>Closing remarks and thanks.</td></tr></tbody></table>

## Mind map

```mermaid
mindmap
  root(( "Graph Neat Presentation" ))
    title("Title: Graph Neat")
    author("Anton Antonov")
    date("November 13th, 20124")
    examples
      example1("Undirected Triangular Grid Graph")
        vertices("Vertices colored by degree")
        matrices
          adjacency("Adjacency Matrix (symmetric)")
          incidence("Incidence Matrix (vertex-edge)")
      example2("Grid Graph Bipartite")
        bipartite("Two sets of vertices")
        code("Raku code generates and verifies bipartite")
        coloring("Parted coloring and highlight lists")
        vertex_coords("Uses known vertex coordinates")
      example3("Directed Grid Graph")
        random_direction("Randomly directed from undirected")
        connected_components("Connected components colored")
        explanation("Reachability defines components")
        visualization
          javascript("JS D3 visualization in Jupyter")
          dot("DOT language SVG visualization")
          shapes("Random vertex/node shapes")
      example4("Collage of Star Graphs")
        description("40 star graphs with 8-16 rays")
        coloring("Subset colored")
        code_origin("Code generated by large language models")
        visualization
          dot("More regular layout with DOT")
          dot_language("DOT language DSL for styling and plots")
    tools
      raku("Raku programming language")
      graph_package("Graph package in Raku")
      visualization
        d3("JavaScript D3 library")
        dot("Graphviz DOT language and tool")
      environment
        jupyter("Jupyter notebook and Raku chatbook")
```

## Thinking hats

<table border="1" cellpadding="5" cellspacing="0">
  <tr>
    <th>Hat</th>
    <th>Perspective</th>
    <th>Feedback</th>
    <th>New Ideas</th>
  </tr>
  <tr>
    <td>White Hat</td>
    <td>Information and Facts</td>
    <td>The presentation covers fundamental graph theory concepts such as undirected graphs, bipartite graphs, adjacency and incidence matrices, connected components in directed graphs, and visualization techniques using Raku and JavaScript libraries like D3 and Graphviz’s dot language. The examples demonstrate the use of Raku's graph package and integration with web technologies for rendering. The explanation of graph properties (degree, bipartite nature) is accurate and well linked to visual representations. The use of random direction assignment and connected component analysis aligns with standard computational graph analysis approaches.</td>
    <td>Include performance metrics or complexity analysis for graph operations, Provide comparative data on visualization libraries (D3 vs dot), Add more detailed documentation on Raku graph package methods</td>
  </tr>
  <tr>
    <td>Black Hat</td>
    <td>Judgement and Caution</td>
    <td>The presentation is somewhat verbose and occasionally unclear, with some disorganized narration ("I don’t want to do that", "um", "so") that may reduce professionalism and audience engagement. The reliance on random direction assignment for directed graphs may lead to non-reproducible results. The approach to visualization using multiple technologies (JavaScript, SVG, dot) could confuse users unfamiliar with these tools. The integration with Jupyter notebooks and chatbooks might limit accessibility if users lack setup knowledge. Also, the use of large language models for code generation without validation could introduce errors.</td>
    <td>Streamline and script the presentation for clarity, Provide reproducible examples with fixed seeds for randomness, Offer setup guides for required environments</td>
  </tr>
  <tr>
    <td>Gray Hat</td>
    <td>Cynicism and Skepticism</td>
    <td>This presentation feels like a showcase of "look how many buzzwords and tools I can throw together" rather than a focused tutorial. The casual language and filler words suggest it might be more of a personal demo than a polished resource. The claim that the code was generated by LLMs but not shown hints at an attempt to impress without transparency. The preference for "vertex" over "node" is a minor ego stroke disguised as a technical choice. Also, the reliance on dot language’s "ease" is subjective and may be overstated.</td>
    <td>Disclose LLM code generation explicitly with examples, Adopt a consistent terminology glossary, Provide a more structured and formal walkthrough</td>
  </tr>
  <tr>
    <td>Red Hat</td>
    <td>Feelings and Emotion</td>
    <td>The presentation feels enthusiastic and passionate about graph theory and Raku programming. The speaker’s excitement about the "neatest example" and the visual appeal of colorful graphs is contagious. However, some hesitation and filler words might make the delivery feel less confident. The use of live coding and interactive visualizations likely makes the session engaging and fun, sparking curiosity and interest in the audience. The personal touch (mentioning the date, name) adds warmth but might distract from content focus.</td>
    <td>Practice delivery to boost confidence, Incorporate audience interaction or Q&A, Use storytelling to connect concepts emotionally</td>
  </tr>
  <tr>
    <td>Yellow Hat</td>
    <td>Benefits and Creativity</td>
    <td>This presentation effectively demonstrates the power of combining Raku’s graph processing with modern visualization tools, opening creative ways to explore complex graph properties interactively. The modular approach (using dot language, JavaScript, SVG) offers flexibility for different user preferences. Showcasing bipartite graphs and connected components visually aids understanding and could inspire educational tools. The collage of star graphs is a creative way to visualize multiple graph instances simultaneously, which could be extended to other graph families.</td>
    <td>Create interactive tutorials for teaching graph theory, Develop a gallery of graph visualizations for educational use, Extend examples to dynamic or weighted graphs</td>
  </tr>
  <tr>
    <td>Green Hat</td>
    <td>Profitability and Potential</td>
    <td>This idea has commercial potential in educational technology, providing interactive graph visualization tools for programming and discrete math courses. The integration with Jupyter notebooks and web technologies suggests easy scalability as an online platform or plugin. The use of Raku, though niche, could attract a dedicated developer community. Additionally, automated code generation via LLMs to produce graph examples could be monetized as a productivity tool for educators and researchers.</td>
    <td>Develop a SaaS platform for interactive graph visualization and teaching, Package the visualization tools as plugins for popular IDEs or notebooks, Offer consulting or workshops on Raku graph programming</td>
  </tr>
  <tr>
    <td>Blue Hat</td>
    <td>Usefulness and Opportunities</td>
    <td>The approach is highly useful for learners and developers exploring graph algorithms and visualization. Opportunities exist to expand this by integrating real-world datasets (social networks, transport maps) to demonstrate practical applications. The combination of Raku and JavaScript visualization in Jupyter notebooks is a compelling environment for experimentation. Further, creating seamless workflows between graph creation, analysis, and visualization would enhance productivity and adoption.</td>
    <td>Integrate real-world graph datasets for demonstration, Build workflow automation between graph generation and visualization, Add support for interactive graph editing and manipulation</td>
  </tr>
  <tr>
    <td>Purple Hat</td>
    <td>Assessment and Direction</td>
    <td>Summarizing, the presentation offers solid technical content (White) but needs refinement in delivery and clarity (Black). It risks coming off as unfocused or self-promotional (Gray), though it conveys genuine enthusiasm (Red). The creative use of visualization is promising (Yellow), with clear commercial and practical potential (Green and Blue). The best path forward is to polish the narrative, ensure reproducibility, and expand interactive capabilities while building a structured educational or commercial product around the tools demonstrated.</td>
    <td>Refine presentation script and polish delivery, Develop reproducible and documented example sets, Explore productization or educational platform development</td>
  </tr>
</table>

---

## Partial evaluation

Drop the results in `LLM::Graph` computed above:

In [11]:
$gCombinedSummary.clear

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

Here the are normalized nodes without results:

In [12]:
.say for |$gCombinedSummary.nodes

Title => {eval-function => sub { }, spec-type => (Callable)}
Summary => {eval-function => sub { }, spec-type => (Routine), wrapper => Routine::WrapHandle.new}
IngestText => {eval-function => sub { }, spec-type => (Callable)}
ThinkingHats => {eval-function => sub { }, spec-type => (Routine), wrapper => Routine::WrapHandle.new}
TopicsTable => {eval-function => sub { }, spec-type => (Routine), wrapper => Routine::WrapHandle.new}
ExportAndOpen => {eval-function => sub { }, spec-type => (Callable), test-function => -> $export-and-open = Bool::True { #`(Block|5522590062304) ... }}
TypeOfInput => {eval-function => sub { }, spec-type => (Routine), wrapper => Routine::WrapHandle.new}
MindMap => {eval-function => sub { }, spec-type => (Routine), wrapper => Routine::WrapHandle.new}
Report => {eval-function => sub { }, spec-type => (Callable)}


Here all results are pre-assigned as arguments:

In [13]:
$gCombinedSummary.eval($url, 
with-title => '«Graph» neat examples, set 3', 
:export-and-open,
Summary => 'In brief',
TypeOfInput => 'Other',
IngestText => 'Ingest text',
ThinkingHats => 'Thinking hats',
TopicsTable => '["TopicsTable"]',
MindMap => 'Mind-map graph',
)

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

-----

## References

### Blog posts

[AA1] Anton Antonov,
["Parameterized Literate Programming"](https://rakuforprediction.wordpress.com/2025/06/21/parameterized-literate-programming/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

### Notebooks

[AAn1] Anton Antonov,
["LLM comprehensive summary template for large texts"](https://community.wolfram.com/groups/-/m/t/3448842),
(2025),
[Wolfram Community](https://community.wolfram.com).

### Packages

[AAp1] Anton Antonov, 
[LLM::Graph, Raku package](https://github.com/antononcube/Raku-LLM-Graph),
(2025),
[GitHub/antononcube](https://github.com/antononcube).